# 3. Blog App
> 기본 기능
- 글 등록
- 열람
- 태그 달기
- 댓글 및 검색 기능
- 콘텐츠 생성 및 편집 기능

## 3.2. 개발 코딩하기
### 3.2.1. 뼈대만들기
- python manage.py startapp blog

In [ ]:
# apps.py : 자동으로 생김
class BlogConfig(AppConfig):
    name = 'blog'

# settings.py
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'bookmark.apps.BookmarkConfig',,
    'blog.apps.Blogconfig'
]

### 3.2.2. 모델 코딩하기

In [ ]:
# models.py
class Post(models.Model):
    title = models.CharField(verbose_name = 'TITLE', max_length = 50)
    # 슬러그: 포스트를 설명하는 핵심 단어 집합, pk로 주로 사용
    slug = models.SlugField('SLUG', unique = True, allow_unicode = True, help_text = 'one word for title alieas')
    description = models.CharField('DESCRIPTION', max_length = 100, blank = True, help_text = 'simple descrition text')
    content = models.TextField('CONTENT')
    create_dt = models.DateTimeField('CREATE DATE', auto_now_add=True)
    modify_dt = models.DateTimeField('MODIFY DATE', auto_now = True)

    class Meta:
        verbose_name = 'post'
        verbose_name_plural = 'posts'
        db_table = 'blog_posts'
        ordering = ('-modify_dt',)

    def __str__(self):
        '''
        객체의 문자열을 객체.title 속성으로 표시
        '''
        return self.title

    def get_absolute_url(self):
        '''
        메소드가 정의된 객체를 지칭하는 url 반환
        '''
        # reverse: url 패턴을 만들어주는 함수
        return reverse('blog:post_detail', args=(self.slug,))

    def get_previous(self):
        '''
        modify_dt컬럼을 기준으로 최신 포스트를 반환하는 함수
        '''
        return self.get_previous_by_modify_dt()

    def get_next(self):
        '''
        modyfy_dt 컬럼을 기준으로 다음 ㅍ포스트를 반환
        '''
        return self.get_next_by_modify_dt()

In [ ]:
# admin.py
from django.contrib import admin
from blog.models import Post
# Register your models here.

@admin.register(Post)
class Postadmin(admin.ModelAdmin):
    '''
    Post 클래스가 admin사이트에서 어떻게 보여줄지 정의
    '''
    # 화면에 출력할 내용
    list_display = ('id', 'title', 'modify_dt')
    # 필더 사이트바
    list_fliter = ('modify_dt')
    # 검색박스 : 어떤 컬럼에서 검색할지 표시    
    search_filed = ('title', 'content')
    #  titie 필드를 사용해러 slug필드가 미리 채워지도록 하기
    prepopulated_fields = {'slug': ('title',)}



#### db에 적용
- python manage.py makemigrations 
- python manage.py migrate

![3-1](images/3-1.png)

### 3.2.4. 뷰 코딩하기

In [ ]:
from django.shortcuts import render
from django.views.generic import ListView, DetailView
from django.views.generic.dates import ArchiveIndexView, YearArchiveView, MonthArchiveView, DayArchiveview, TodayArchiveview
from blog.models import Post

# Create your views here.

# ListView
class PostLV(ListView):
    model = Post
    template_name = 'blog/post_all.html'
    context_object_name = 'posts'
    paginate_by = 2

#DetailView
class PostDV(DetailView):
    model = Post

# ArchiveView
class PostAV(ArchiveIndexView):
    model = Post
    date_field = 'modify_dt'

class PostYAV(YearArchiveView):
    model = Post
    date_filed = 'modefy_dt'
    make_object_list = True

class PostMAV(MonthArchiveView):
    model = Post
    date_field = 'modity_dt'

class PostDAV(DayArchiveView):
    model = Post
    date_field = 'modify_dt'

class PostTAv(TodayArchiveview):
    model = Post
    Dete_field = 'modify_dt'

### 3.2.5. 템플릿 코딩하기

In [ ]:
#post_all.html
<h1>Blog List</h1>
<br>

{% for post in posts %}
    <h3><a href = '{{post.get_absolute_url}}'>{{post.title}}</a></h3>
    {{post.modify_dt|date:"N d, Y"}}
    <p>{{post.description}}</p>
{% endfor %}

<br>

<div>
    <span>
        <!-- page_obj: 장고의 page객체가 들어있는 context 변수 -->
        {% if page_obj.has_previous %}
            <a href= "?page = {{page_obj.previous_pate_number}}">PreviousPage</a>
        {% endif %}

        Page {{page_obj.number}} of {{page_obj.pagiinator.num_pages}}

        {% if page_obj.has_next %}
            <a href="?page = {{page_obj.next_page_number}}">NextPage</a>
        {% endif %}
    </span>

</div>

#### 코드 설명
> posts객체 (= object_list)
- PostLV 클래스 뷰에서 넘겨주는 컨텍스트 변수
- Post 객체 리스트가 담겨있음.
- post.title: 제목 
- post.description
- post.modify_dt

> post.modify_dt|date:"N d, Y"
- 날짜 형식을 N d, Y포멧으로 출력

> page_obj
- Page 객체가 들어있는 컨텍스트 변수
- 현재 페이지를 기준으로 이전 페이지가 있는지를 확인

> page_obj.previous_page_number
- 이전 페이지의 번호

> page_obj.number
- 현재 페이지 번호

> page_obj.paginator.num_pages
- 총 페이지 개수

#### 템플릿에서 url 추출 함수
> get_absolute_url()
- 간접적으로 url 패턴명을 사용
- 모델 클래스의 메소드로 정의되어 있어야 사용 가능
- reverse()함수를 꼭 사용해햐 함
- {{post.get_absolute_url}}

> {% url %}
- 직접 태그의 인자로 url 패턴명 사용
- {{% url 'blog:post_detail' post.slug %}} 


![3-2](images/3-2.png)

In [ ]:
# post_detail.html
<h2>{{object.title}}</h2>

<p>
{% if object.get_previous %}
    <a href = "{{object.get_previous.get_apsolute_url}}" title = "View previous post">&laquo;-{{object.get_previous}}</a>
{% endif %}

{% if object.get_next %}
    <a href = '{{object.get_next.get_apsolute_url}}'title = "View next post">-&raquo;</a>
{% endif %}

</p>

<p>{{object.modify_dt|date:"j F Y"}}</p>
<br>

<div>
    {{object.content|linebreaks}}
</div>

#### 코드 설명
> object 객체
- PostDV클래스형 뷰에서 컨텍스트 변수로 넘겨주는 Post클래스의 객체

> get_previous 함수
- modify_dt 컬럼 기준으로 이전 객체 반환
- 변경 날짜가 현재 객체보다 더 최신 객체가 있는지 확인

> &laquo;
- html 특수문자(<<)

![3-3](images/3-3.png)

### 3.2.6. 날짜 제네릭뷰와 관련된 템플릿 파일
> 디폴드 템플릿 파일명
- 모델클래스소문자_archive.html

In [ ]:
# post_archive.html
# 포스트 리스트를 날자별로 구분해서 보여주는 화면
